## Part 1

*Условие:
происходят рандомные перестановки P.
Когда оптимальное количество свопов начинает различаться с |P|?*

Ниже приведен код, которые считает d - минимальное количество swap,
чтобы вернуться в сортированное состояние

In [4]:
def in_its_place(elem_indexed, i):
    return elem_indexed == i

def d_min_swap(permut):
    sz = len(permut)
    indexed_permut = [*enumerate(permut)]
    indexed_permut = sorted(indexed_permut, key=lambda e: e[1])
    visited = {b: False for b in range(sz)}
    ret = 0
    for i in range(sz):
        if in_its_place(indexed_permut[i][0], i) or visited[i]:
            continue
        cycle_len = 0
        traverse = i
        while not visited[traverse]:
            visited[traverse] = True
            traverse = indexed_permut[traverse][0]
            cycle_len += 1
        if cycle_len > 0:
            ret += cycle_len - 1
    return ret

print(d_min_swap([2, 4, 1, 5, 3]))

4


В качестве оценки беспорядка выступает инвариант количества циклов.
В сортированном массиве длины n будет n циклов

<img src="pictures/permut1.jpg">
<img src="pictures/permut2.jpg">

При помощи алгоритма был получен график, где:

**синий**: y = x, т.е было x рандомных свапов и точно хватит столько (оценка сверху), чтобы вернуться

**красный**: f(x), пока не вывел теоретически зависимость, возможно стоит пологарифмировать данные

**зеленый**: дельта между верхней оценкой (синий) и идеальным решением (красный)

**черный**: количество циклов, инвариант

<img src="plots/part1_all">


## Part 2

*Условие: изучить Cooler API на примере
ftp://cooler.csail.mit.edu/coolers/hg19/Rao2014-IMR90-MboI-allreps-filtered.500kb.cool*

Для удобства выведем полезную информацию

In [ ]:
import cooler
import math

In [ ]:
def helper(filepath):
    c = cooler.Cooler(filepath)
    print(f'c.info\n{c.info}\n')
    bin_size = 500000 # from c.info
    print(f'c.chromsizes\n{c.chromsizes}\n')
    count_bins_chr1 = math.ceil(c.chromsizes[0] / bin_size)
    print(f'bins in chr1:\n{count_bins_chr1}\n')

helper(filepath='data/Rao2014-IMR90-MboI-allreps-filtered.500kb.cool')

Отдельно выводим информацию по chr1, так как в дальнейшем будем работать именно с ней.
Также интересно посмотреть на hi-c распределение

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def heatmap_hic(filepath):
    c = cooler.Cooler(filepath)
    bin_size = 500000
    count_bins_chr1 = math.ceil(c.chromsizes[0] / bin_size)
    contact_matrix = np.array(c.matrix(balance=False)[:count_bins_chr1, :count_bins_chr1])
    plt.title('hic')
    plt.imshow(contact_matrix, cmap='hot', interpolation='nearest')
    plt.show()

heatmap_hic(filepath='data/Rao2014-IMR90-MboI-allreps-filtered.500kb.cool')

График получился не очень наглядным, потому что порядок k близких к главной диагонали в разы превышает все остальные случаи

**N.B:** здесь и в дальнейшем под k подразумевается расстояние в бинах между i и i + k (в линейном представлении) участками

Для задачи 3 потребуется добыть из chr1 зависимости:

**E(k):** матожидание количества попавших в каплю участков на расстоянии k

* **N.B:** так как матрица очевидно симметричная (расстояние для [i][j] и [j][i] участков одинаковое), то будем рассматривать
только все пары (i, i + k) над главной диагональю

**D(k), SIGMA(k):** аналогичным образом извлекаются из верхней диагонали матрицы

**Гистограмма**: понадобится при построении hi-c в части 3, чтобы не брать только E(k)


Итак, приступим к получению нужных параметров!!

* код для построения графиков опущу, можно посмотреть здесь:
https://github.com/Timoniche/BioInf/blob/main/main_cool_part2.py

**E(k)**
<img src="plots/chr1_EX_k_1_10">

можно заметить, что после k > 2 происходит резкий спад порядки,
тогда в дальнейшем при построении графиков k = 1..10 можно будет опускать (или логарифмировать и смотреть за порядком)

<img src="plots/chr1_EX_k_10_25">

**D(k)**
<img src="plots/chr1_DX_k_1_10">
<img src="plots/chr1_DX_k_10_25">

**SIGMA(k)**
<img src="plots/chr1_SIGMA_k_1_10">
<img src="plots/chr1_SIGMA_k_10_25">

**Гистограмма**

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
def get_upper_k_diagonal(mat, k):
    n = len(mat)
    ret = []
    i = 0
    for j in range(k, n):
        ret.append(mat[i][j])
        i += 1
        j += 1
    return ret

def histogramm(filepath, k=11):
    c = cooler.Cooler(filepath)
    bin_size = 500000
    count_bins_chr1 = math.ceil(c.chromsizes[0] / bin_size)
    contact_matrix = c.matrix(balance=False)[:count_bins_chr1, :count_bins_chr1]
    diag = get_upper_k_diagonal(contact_matrix, k)
    hist = np.histogram(np.array(diag))
    plt.hist(hist)


histogramm(filepath='data/Rao2014-IMR90-MboI-allreps-filtered.500kb.cool')

